# Morisita-Horn similarity calculation



In [2]:
from __future__ import print_function

from collections import Counter
from datetime import datetime
import itertools
import multiprocessing as mp
import os
import subprocess as sp
import sys
import tempfile
import time

import numpy as np
import pandas as pd

from abutils.utils.jobs import monitor_mp_jobs
from abutils.utils.pipeline import list_files, make_dir
from abutils.utils.progbar import progress_bar

### User-defined options

By default, the size of the bootstrap samples will increase exponentially, as the similarity plots will be drawn with a logarithmic x-axis. However, the option is also given to draw bootstrap samples that increase linearly in size rather than exponentially. The following options may be adjusted depending on the desired output:

  * `iterations` is the number of replicate samplings for each subsample size. Default is `10`.  
  
  * `max_power_of_10` is the highest exponent of 10 for which subsamples will be drawn. For example, the default value of `7` means that the largest bootstrap sample will be `10^7`, or 10 million, sequences. The lowest acceptable value is `2`, as subsampling fewer than 100 sequences is not especially useful.  
  
  * `subsample_fraction` is the fraction of each `power_of_10` multiple at which the bootstrap sample size increases. For example, the default value of `0.3` results in the following multipliers: `[1.0, 1.3, 1.6, 1.9]`. For a `power_of_10` of 10^6, for example, a `subsample_fraction` of `0.3` would result in the following bootstrap sample sizes: `1.0x10^6, 1.3x10^6, 1.6x10^6, and 1,9x10^6`.  
  
  * `subsample_size` is the size multiple for each subsample. By default (if `subsample_fraction` is provided), this will not be used. This option is only provided in case you would prefer the subsample size pools to increase in linear fashion, rather than exponentially.

Note that the data directory (`'./data/techrep-merged_vj-cdr3len_no-header/'`) is not present in this Github repo, as the size of the files far exceeds what is allowed by Github. You can download a compressed archive containing the appropriate data files [**here**](http://burtonlab.s3.amazonaws.com/GRP_github_data/techrep-merged_vj-cdr3len_no-header.tar.gz). Decompressing the archive inside of `./data` (the "data" directory found in the same parent directory as this notebook) should allow you to run the following code without alteration.


In [3]:
iterations = 10
max_power_of_10 = 7
subsample_fraction = 0.3
subsample_size = 25000

data_dir = './data/techrep-merged_vj-cdr3len_no-header/'
temp_dir = './data/temp/'
output_dir = './data/user-calculated_mh_similarity/'

### Subjects and directories

In [4]:
individual_files_dir = os.path.join(output_dir, 'individual_comparisons')
make_dir(temp_dir)
make_dir(output_dir)
make_dir(individual_files_dir)

with open('../data_processing/data/subjects.txt') as f:
    subjects = sorted(f.read().split())

### Morisita-Horn similarity

In [4]:
def mh_similarity(sample1, sample2):
    '''
    Calculates the Marista-Horn similarity for two samples.

    .. note:

        sample1 and sample2 should be the same length, and
        the sum of each sample should be greater than 0.

    Args:

        sample1 (list): list of frequencies for sample 1
        
        sample2 (list): list of frequencies for sample 2

    Returns:

        float: Marista-Horn similarity (between 0 and 1)
    '''
    X = sum(sample1)
    Y = sum(sample2)
    XY = X * Y
    sumXiYi = 0
    sumXiSq = 0
    sumYiSq = 0
    for x, y in zip(sample1, sample2):
        sumXiYi += x * y
        sumXiSq += x * x
        sumYiSq += y * y
    num = 2 * sumXiYi
    denom = (float(sumXiSq) / (X * X) + float(sumYiSq) / (Y * Y)) * XY
    return 1. * num / denom


def load_data(files):
    '''
    Loads VJ-CDR3len data from a list of files corresponding to a single subject.
    
    Args:
    
        files (list): a list of files containing data to load. Data will be
                      loaded for all files and returned as a single list.
    
    Returns:
    
        list: a combined list of data from all of the files
    '''
    data = []
    for f in files:
        with open(f) as of:
            for line in of:
                _d = '_'.join(line.strip().split())
                if _d:
                    data.append(_d)
    return data


def get_subsample_sizes():
    '''
    Returns a list of subsample sizes, based on user-defined subsampling options.
    '''
    if subsample_fraction is not None:
        sizes = []
        for mpt in range(2, max_power_of_10 + 1):
            start = 10**(mpt - 1)
            end = 10**mpt
            step = int(10**mpt * float(subsample_fraction))
            sizes += list(range(start, end, step))
        sizes.append(10**mpt)
    else:
        sizes = range(subsample_step, 10 ** max_power_of_10, subsample_step)
    return sizes


def compute_frequencies(data, iterations, size):
    '''
    Subsamples a dataset (with replacement) and computes the VJ-CDR3len
    frequency for each bootstrap sample.
    
    Args:
    
        data (list): a list of antibody sequences collapsed to just VJ-CDR3len
        
        iterations (int): the number of bootstrap samplings to be performed
        
        size (int): the size (in sequences) of each bootstrap sample
    
    Returns:
    
        list(Counter): a list of VJ-CDR3len frequencies (as Counter objects)
    '''
    subsamples = np.random.choice(data, size=(iterations, size), replace=True)
    freqs = []
    for subsample in subsamples:
        freqs.append(Counter(subsample))
    return freqs


def compute_similarity_for_single_size(sub1_data, sub2_data, iterations, size):
    '''
    For a single bootstrap sampling size, computes Morisita-Horn similarity of
    two datasets.
    
    Args:
    
        sub1_data (list): a list of VJ-CDR3len values
        
        sub2_data (list): a list of VJ-CDR3len values
        
        iterations (int): the number of iterations to be performed
        
        size (int): size (in sequences) of each bootstrap sampling
        
    Returns:
    
        int: the size of each bootstrap sampling
        
        list: a list of Morisita-Horn similarities, of length `iterations`
    '''
    similarities = []
    sub1_freqs = get_frequencies(sub1_data, iterations, size)
    sub2_freqs = get_frequencies(sub2_data, iterations, size)
    for s1, s2 in zip(sub1_freqs, sub2_freqs):
        freq_df = pd.DataFrame({'sub1': s1, 'sub2': s2}).fillna(0)
        similarities.append(mh_similarity(freq_df['sub1'], freq_df['sub2']))
    return size, similarities


def calculate_similarities(subject1, subject2, iterations, sizes):
    '''
    Performs Morisita-Horn similarity calculations on VJ-CDR3len data for two subjects.
    
    Args:
    
        subject1 (str): name of subject 1
        
        subject2 (str): name of subject 2
        
        iterations (int): number of iterations to be performed for each bootstrap sample size
        
        sizes (list(int)): a list of bootstrap sample sizes
        
    Returns:
    
        sub_header (str): a header line containing subject information
        
        similarities (dict): similarity scores, with the dict keys being sample sizes
                             and values being lists of similarity scores of length `iterations`
    '''
    sub_header = '#{} {}'.format(subject1, subject2)
    sub1_dir = os.path.join(data_dir, subject1)
    sub2_dir = os.path.join(data_dir, subject2)
    sub1_files = list_files(sub1_dir)
    sub2_files = list_files(sub2_dir)
    similarities = {}
    output_data = [sub_header, ]
    output_file = os.path.join(individual_files_dir, '{}-{}'.format(subject1, subject2))
    
    # load all of the files into memory
    sub1 = os.path.basename(os.path.dirname(sub1_files[0]))
    sub1_data = load_data(sub1_files)
    sub2 = os.path.basename(os.path.dirname(sub2_files[0]))
    sub2_data = load_data(sub2_files)
    
    for size in sizes:
        similarities[size] = []
        sub1_freqs = compute_frequencies(sub1_data, iterations, size)
        sub2_freqs = compute_frequencies(sub2_data, iterations, size)
        for s1, s2 in zip(sub1_freqs, sub2_freqs):
            freq_df = pd.DataFrame({'sub1': s1, 'sub2': s2}).fillna(0)
            similarities[size].append(mh_similarity(freq_df['sub1'], freq_df['sub2']))
        output_data.append(' '.join([str(v) for v in [size] + similarities[size]]))
        with open(output_file, 'w') as f:
            f.write('\n'.join(output_data))
    return sub_header, similarities

### Calculate similarity

Morisita-Horn similarity will be calculated for each pairwise combination of subjects (including self-comparisons). The process is multiprocess (one pairwise comparison per process) and will use as many cores as necessary to perform all comparisons (there are a total of 55 comparisons from 10 subjects, so the max number of cores that will be used is 55).

In [5]:
# get a list of subsample sizes, based on user-defined options
sizes = get_subsample_sizes()

# get a list of all pairwise combinations of subjects (including self-comparison)
combinations = list(itertools.combinations_with_replacement(subjects, 2))

p = mp.Pool(processes=7, maxtasksperchild=1)
start = datetime.now()
async_results = []

# initialize the progress bar
jobs = len(combinations)
progress_bar(0, jobs, start_time=start)

# calculate the similarity score for each pairwise combination of subjects
for subject1, subject2 in combinations:
    async_results.append(p.apply_async(calculate_similarities, args=(subject1, subject2, iterations, sizes)))
monitor_mp_jobs(async_results, start_time=start)
results = [ar.get() for ar in async_results]

p.close()
p.join()

(0/55) |                                                  |  0%  (11:02)  

Process ForkPoolWorker-6:
KeyboardInterrupt
Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-7:
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
  File "<ipython-input-4-1994935e457c>", line 168, in calculate_similarities
    sub2_freqs = compute_frequencies(sub2_data, iterations, size)
  File "<ipython-input-4-1994935e457c>", line 92, in compute_frequencies
    subsamples = np.random.choice(data, size=(iterations, size), replace=True)
  File "mtrand.pyx", line 1112, in mtrand.RandomState.choice
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6

KeyboardInterrupt: 

### Combine similarity files

Each pairwise comparison resulted in a separate output file. Here we combine them into a single similarities file.

In [35]:
combined_output_file = os.path.join(output_dir, 'mh-similarities_combined.txt')
individual_files = list_files(individual_files_dir)

with open(combined_output_file, 'w') as f:
    f.write('')

cat_cmd = 'for f in {}/*; do (cat "${{f}}"; echo) >> {}; done'.format(individual_files_dir.rstrip('/'), combined_output_file)
p = sp.Popen(cat_cmd, stdout=sp.PIPE, stderr=sp.PIPE, shell=True)
stdout, stderr = p.communicate()

In [5]:
for s in subjects:
    print(s)

316188
326650
326651
326713
326737
326780
326797
326907
327059
D103
